In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [19]:
# Export model
def export_onnx(model, data_tensor_array, model_loc):
  circuit = model()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  # print(device)

  circuit.to(device)

  # Flips the neural net into inference mode
  circuit.eval()
  input_names = []
  dynamic_axes = {}

  data_tensor_tuple = ()
  for i in range(len(data_tensor_array)):
    data_tensor_tuple += (data_tensor_array[i],)
    input_index = "input"+str(i+1)
    input_names.append(input_index)
    dynamic_axes[input_index] = {0 : 'batch_size'}
  dynamic_axes["output"] = {0 : 'batch_size'}

  # Export the model
  torch.onnx.export(circuit,               # model being run
                      data_tensor_tuple,                   # model input (or a tuple for multiple inputs)
                      model_loc,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes=dynamic_axes)

In [20]:
# mode is either "accuracy" or "resources"

def gen_settings(comb_data_path, onnx_filename, scale, mode, settings_filename):
  print("==== Generate & Calibrate Setting ====")
  # Set input to be Poseidon Hash, and param of computation graph to be public
  # Poseidon is not homomorphic additive, maybe consider Pedersens or Dory commitment.
  gip_run_args = ezkl.PyRunArgs()
  gip_run_args.input_visibility = "hashed"  # matrix and generalized inverse commitments
  gip_run_args.output_visibility = "public"   # no parameters used
  gip_run_args.param_visibility = "private" # should be Tensor(True)--> to enforce arbitrary data in w

 # generate settings
  ezkl.gen_settings(onnx_filename, settings_filename, py_run_args=gip_run_args)
  if scale =="default":
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode)
  else:
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode, scales = scale)

  assert os.path.exists(settings_filename)
  assert os.path.exists(comb_data_path)
  assert os.path.exists(onnx_filename)
  f_setting = open(settings_filename, "r")
  print("scale: ", scale)
  print("setting: ", f_setting.read())


In [21]:
def verifier_define_calculation(verifier_model, verifier_model_path, dummy_data_path_array):
  # load data from dummy_data_path_array into dummy_data_array
  dummy_data_tensor_array = []
  # comb_dummy_data = []
  for path in dummy_data_path_array:
    dummy_data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    dummy_data_tensor_array.append(torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 )))
    # comb_dummy_data.append(dummy_data.tolist())
  # export onnx file
  export_onnx(verifier_model,dummy_data_tensor_array, verifier_model_path)

In [22]:
# we decide to not have comb_data_path as parameter since a bit redundant parameter.
def prover_gen_settings(data_path_array, verifier_model_path, scale, mode, settings_path):
    comb_data = []
    for path in data_path_array:
        data = np.array(json.loads(open(path, "r").read())["input_data"][0])
        comb_data.append(data.tolist())
    # this is private to prover since it contains actual data
    comb_data_path = os.path.join('prover/comb_data.json')
    # Serialize data into file:
    json.dump(dict(input_data = comb_data), open(comb_data_path, 'w' ))

    # gen + calibrate setting
    gen_settings(comb_data_path, verifier_model_path, scale, mode, settings_path)

In [23]:
# Here prover can concurrently call this since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
def verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path ):
  # compile circuit
  res = ezkl.compile_circuit(verifier_model_path, verifier_compiled_model_path, settings_path)
  assert res == True

  # srs path
  res = ezkl.get_srs(srs_path, settings_path)

  # setupt vk, pk param for use..... prover can use same pk or can init their own!
  print("==== setting up ezkl ====")
  start_time = time.time()
  res = ezkl.setup(
        verifier_compiled_model_path,
        vk_path,
        pk_path,
        srs_path)
  end_time = time.time()
  time_setup = end_time -start_time
  print(f"Time setup: {time_setup} seconds")

  assert res == True
  assert os.path.isfile(vk_path)
  assert os.path.isfile(pk_path)
  assert os.path.isfile(settings_path)

In [24]:
def prover_gen_proof(prover_model, data_path_array, witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path):
  data_tensor_array = []
  comb_data = []
  for path in data_path_array:
    data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    data_tensor_array.append(torch.reshape(torch.tensor(data), (1, len(data),1 )))
    comb_data.append(data.tolist())

  # export onnx file
  export_onnx(prover_model, data_tensor_array, prover_model_path)

  # not necessarily be the same path as in prover_gen_settings, but to make it not confused, make it the same
  comb_data_path = os.path.join('shared/comb_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_data), open(comb_data_path, 'w' ))

  res = ezkl.compile_circuit(prover_model_path, prover_compiled_model_path, settings_path)
  assert res == True
  # now generate the witness file
  print('==== Generating Witness ====')
  witness = ezkl.gen_witness(comb_data_path, prover_compiled_model_path, witness_path)
  assert os.path.isfile(witness_path)
  # print(witness["outputs"])
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']
  print("witness boolean: ", ezkl.vecu64_to_float(witness['outputs'][0][0], output_scale[0]))
  for i in range(len(witness['outputs'][1])):
    print("witness result", i+1,":", ezkl.vecu64_to_float(witness['outputs'][1][i], output_scale[1]))

  # GENERATE A PROOF
  print("==== Generating Proof ====")
  start_time = time.time()
  res = ezkl.prove(
        witness_path,
        prover_compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

  print("proof: " ,res)
  end_time = time.time()
  time_gen_prf = end_time -start_time
  print(f"Time gen prf: {time_gen_prf} seconds")
  assert os.path.isfile(proof_path)

In [25]:
def verifier_verify(proof_path, settings_path, vk_path, srs_path):
  # enforce boolean statement to be true
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']

  proof = json.load(open(proof_path))
  num_inputs = len(settings['model_input_scales'])
  print("num_inputs: ", num_inputs)
  proof["instances"][0][num_inputs] = ezkl.float_to_vecu64(1.0, output_scale[0])
  json.dump(proof, open(proof_path, 'w'))

  print("prf instances: ", proof['instances'])

  print("proof boolean: ", ezkl.vecu64_to_float(proof['instances'][0][num_inputs], output_scale[0]))
  for i in range(num_inputs+1, len(proof['instances'][0])):
    print("proof result",i-num_inputs,":", ezkl.vecu64_to_float(proof['instances'][0][i], output_scale[1]))


  res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

  assert res == True
  print("verified")

In [26]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [27]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)



In [34]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<0.01*torch.sum(X), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [35]:
# prover calculates settings, send to verifier
prover_gen_settings([data_path], verifier_model_path, [0], "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-2332,36068],"logrows":16,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":609,"total_const_size":1,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [36]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

# Prover generates proof
theory_output = torch.mean(data_tensor)
print("theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<0.01*torch.sum(X), self.w)
prover_gen_proof(prover_model, [data_path], witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 5.456499099731445 seconds
theory_output:  tensor(49.9700)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18373208119716085496, 9977853902793730609, 17093442942004944878, 1769661249493325049]]], 'proof': '102cb27f662a8dad9afab83aa018e10e6e5c864e629be9fbfb5bbedea5bb421703aa74c5e58daa0e8436378249eec399a51088d90f6acc6e24f0f55d6034d8d8018d68dbe5a08fe6b77404ef431476226e6ebe75efa56925f4a6a17b094081fc2c88595b48b0644d53222780bf6235158fff657654afb66145e7819d4f0796a11b94cff2537ec437004e1d1f17930a68995ef9ca55c0f77e675dc58bf298253f04cee2c3f66f0df039b776dfd38861adbdfdb9b8c013a5ec800190d99032239814e2c8be7b4e73df9ccfc3d32c74d970e5ca14cdc6b67888b30dfaa8919b653b27831601f1bbc32d988db908f70554c6bc0cafa04b6beace63ac1b8af058293b093d3b77ce9738770a2b934f64cee1bb412f49e71bab5810608315e8f8ac18c315dca7872c09e8859ba1dd41285bdf9b5c27cea4129e9cfb9bbfd5961fc2af722bc791d4fd20a92b3a59aa74255a2d267026cbf8ab1755add962532f4ccf

In [38]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18373208119716085496, 9977853902793730609, 17093442942004944878, 1769661249493325049]]]
proof boolean:  1.0
proof result 1 : 50.0
verified
